In [1]:
import torch
import matplotlib.pyplot as plt
import random

## Callbacks
### Callbacks as GUI events

In [2]:
import ipywidgets as widgets

In [11]:
w = widgets.Button(description='click me')
output = widgets.Output()

In [15]:
def f(o): 
    with output:
        print('Hi')

In [16]:
w.on_click(f)

In [17]:
display(w, output)

Button(description='click me', style=ButtonStyle())

Output(outputs=({'name': 'stdout', 'text': 'Hi\n', 'output_type': 'stream'},))

### Creating your own callback

In [18]:
from time import sleep

In [21]:
def slow_calculation():
    res = 0
    for i in range(5):
        res += i*i
        sleep(1)
    return res

In [23]:
slow_calculation()

30

In [25]:
def slow_calculation(cb=None):
    res = 0
    for i in range(5):
        res += i*i
        sleep(i)
        if cb: cb(i)
    return res

In [28]:
def show_progress(epoch): print(f"Awesome! We've finished epoch {epoch}")

In [29]:
slow_calculation(cb=show_progress)

Awesome! We've finished epoch 0
Awesome! We've finished epoch 1
Awesome! We've finished epoch 2
Awesome! We've finished epoch 3
Awesome! We've finished epoch 4


30

### Lambdas and partials


In [32]:
slow_calculation(lambda o: print(f"Awesome! We've finished epoch {o}"))

Awesome! We've finished epoch 0
Awesome! We've finished epoch 1
Awesome! We've finished epoch 2
Awesome! We've finished epoch 3
Awesome! We've finished epoch 4


30

In [34]:
def show_progress(exclamation, epoch): print(f"{exclamation}! We've finished epoch {epoch}")

In [35]:
slow_calculation(lambda o: show_progress("Ok I guess", o))

Ok I guess! We've finished epoch 0
Ok I guess! We've finished epoch 1
Ok I guess! We've finished epoch 2
Ok I guess! We've finished epoch 3
Ok I guess! We've finished epoch 4


30

In [36]:
def make_show_progress(exclamation):
    def _inner(epoch): print(f"{exclamation}! We've finished epoch {epoch}!")
    return _inner

In [37]:
slow_calculation(make_show_progress('Nice!'))

Nice!! We've finished epoch 0!
Nice!! We've finished epoch 1!
Nice!! We've finished epoch 2!
Nice!! We've finished epoch 3!
Nice!! We've finished epoch 4!


30

In [38]:
from functools import partial

In [43]:
slow_calculation(partial(show_progress, epoch="OK I guess"))

0! We've finished epoch OK I guess
1! We've finished epoch OK I guess
2! We've finished epoch OK I guess
3! We've finished epoch OK I guess
4! We've finished epoch OK I guess


30

In [44]:
f2 = partial(show_progress, "OK I guess")

In [45]:
f2(1)

OK I guess! We've finished epoch 1


### Callbacks as callable classes

In [49]:
class ProgressShowingCallback():
    def __init__(self, exclamation="Awesome"): self.exclamation = exclamation
    def __call__(self, epoch): print(f"{self.exclamation}! We've finished epoch {epoch}")

In [50]:
cb = ProgressShowingCallback("Just Super")

In [51]:
slow_calculation(cb)

Just Super! We've finished epoch 0
Just Super! We've finished epoch 1
Just Super! We've finished epoch 2
Just Super! We've finished epoch 3
Just Super! We've finished epoch 4


30

### Multiple callback funcs: `*args` `**kwargs`

In [52]:
def f(*a, **b): print(f"args: {a}, kwargs: {b}")

In [53]:
f(1,2,3, a="a", b="b")

args: (1, 2, 3), kwargs: {'a': 'a', 'b': 'b'}


In [54]:
def g(a,b,c=0): print(a,b,c)

In [55]:
args = [1,2]
kwargs = {'c': 3}
g(*args, **kwargs)

1 2 3


In [57]:
def slow_calculation(cb=None):
    res=0
    for i in range(5):
        if cb: cb.before_calc(i)
        res += i*i
        if cb: cb.after_calc(i, val=res)
    return res

In [58]:
class PrintStepCallback():
    def before_calc(self, *args, **kwargs): print(f"About to start")
    def after_calc(self, *args, **kwargs): print(f"Done Step")

In [59]:
slow_calculation(PrintStepCallback())

About to start
Done Step
About to start
Done Step
About to start
Done Step
About to start
Done Step
About to start
Done Step


30

In [61]:
class PrintStatusCallback():
    def __init__(self): pass
    def before_calc(self, epoch, **kwargs): print(f"About to start: {epoch}")
    def after_calc(self, epoch, val, **kwargs): print(f"After {epoch}: {val}")

In [62]:
slow_calculation(PrintStatusCallback())

About to start: 0
After 0: 0
About to start: 1
After 1: 1
About to start: 2
After 2: 5
About to start: 3
After 3: 14
About to start: 4
After 4: 30


30

### Modifying behaviour


In [65]:
def slow_calculation(cb=None):
    res=0
    for i in range(5):
        if cb and hasattr(cb, "before_calc"): cb.before_calc(i)
        res += i*i
        sleep(1)
        if cb and hasattr(cb, "after_calc"): 
            if cb.after_calc(i, res):
                print('Stopping Early')
                break
    return res

In [66]:
class PrintAfterCallback():
    def after_calc (self, epoch, val):
        print(f"After {epoch}: {val}")
        if val>10: return True

In [67]:
slow_calculation(PrintAfterCallback())

After 0: 0
After 1: 1
After 2: 5
After 3: 14
Stopping Early


14

In [70]:
class SlowCalculator():
    def __init__(self, cb=None): self.cb, self.res = cb, 0
    def callback(self, cb_name, *args):
        if not self.cb: return
        cb = getattr(self.cb, cb_name, None)
        if cb: return cb(self, *args)
    def calc(self):
        for i in range(5):
            self.callback('before_calc', i)
            self.res += i*i
            sleep(1)
            if self.callback('after_calc', i):
                print("stopping early")
                break

In [71]:
class ModifyingCallback():
    def after_calc(self, calc, epoch):
        print(f"After {epoch}: {calc.res}")
        if calc.res>10: return True
        if calc.res<3: calc.res = calc.res*2

In [72]:
calculator = SlowCalculator(ModifyingCallback())

In [73]:
calculator.calc()

After 0: 0
After 1: 1
After 2: 6
After 3: 15
stopping early


In [74]:
calculator.res

15

## __dunder__ thingies
Anything that looks like __this__ is, in some way, special. Python, or some library, can define some functions that they will call at certain documented times. For instance, when your class is setting up a new object, python will call __init__. These are defined as part of the python data model.

For instance, if python sees +, then it will call the special method __add__. If you try to display an object in Jupyter (or lots of other places in Python) it will call __repr__.

In [77]:
class SloppyAdder():
    def __init__(self,o): self.o=o
    def __add__(self,b): return SloppyAdder(self.o + b.o + 0.01)
    def __repr__(self): return str(self.o)

In [78]:
a = SloppyAdder(1)
b = SloppyAdder(2)
a+b

3.01

Special methods you should probably know about (see data model link above) are:

- __getitem__
- __getattr__
- __setattr__
- __del__
- __init__
- __new__
- __enter__
- __exit__
- __len__
- __repr__
- __str__

## `__getattr__` and getattr

In [79]:
class A: a,b=1,2

In [80]:
a = A()
a.b

2

In [81]:
getattr(a, 'b')

2

In [89]:
getattr(a, 'b' if random.random()>0.5 else 'a')

1

In [91]:
class B:
    a, b = 1, 2
    def __getattr__(self, k):
        if k[0]=='_': raise AttributeError(k)
        return f'Hello from {k}'

In [92]:
b = B()
b.a

1

In [93]:
b.foo

'Hello from foo'

In [94]:
b._fo

AttributeError: _fo